# NYC Taxi Data - Forecasting Models

Building and comparing different forecasting models for NYC taxi demand prediction.

In [ ]:
import sys
import os
sys.path.append(os.path.join('..', 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils.data_loader import load_time_series, create_time_features
from utils.preprocessing import create_lag_features, create_rolling_features, scale_data
from models.forecasting import (
    NaiveForecaster, SeasonalNaiveForecaster, ARIMAForecaster, 
    ExponentialSmoothingForecaster, RandomForestForecaster, 
    EnsembleForecaster, ModelSelector
)
from visualization.plots import plot_forecast_comparison, plot_residuals

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Data Loading and Preprocessing

In [ ]:
# Load NYC taxi data
df = pd.read_csv('../data/raw/nyc_taxi.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp')

print(f"Data shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")

# Basic statistics
print(f"\nBasic Statistics:")
print(df['value'].describe())

In [ ]:
# Create time-based features
df_features = create_time_features(df)
print(f"Features added: {[col for col in df_features.columns if col != 'value']}")

In [ ]:
# Add lag and rolling features for ML models
df_ml = create_lag_features(df_features, 'value', lags=[1, 2, 3, 24, 48, 168])
df_ml = create_rolling_features(df_ml, 'value', windows=[3, 12, 24, 48])

print(f"Total features created: {len(df_ml.columns)}")
print(f"Feature columns: {list(df_ml.columns)}")

## 2. Train/Test Split

In [ ]:
# Time series split (last 20% for testing)
split_ratio = 0.8
split_point = int(len(df) * split_ratio)

# For basic models (only need target variable)
train_data = df['value'][:split_point]
test_data = df['value'][split_point:]

# For ML models (need features)
df_ml_clean = df_ml.dropna()  # Remove rows with NaN from lag features
ml_split_point = int(len(df_ml_clean) * split_ratio)

train_ml = df_ml_clean[:ml_split_point]
test_ml = df_ml_clean[ml_split_point:]

print(f"Training period: {train_data.index[0]} to {train_data.index[-1]}")
print(f"Test period: {test_data.index[0]} to {test_data.index[-1]}")
print(f"Training samples: {len(train_data):,}")
print(f"Test samples: {len(test_data):,}")
print(f"ML training samples: {len(train_ml):,}")
print(f"ML test samples: {len(test_ml):,}")

In [ ]:
# Visualize train/test split
plt.figure(figsize=(15, 6))
plt.plot(train_data.index, train_data.values, label='Training Data', color='blue', alpha=0.7)
plt.plot(test_data.index, test_data.values, label='Test Data', color='red', alpha=0.7)
plt.axvline(x=train_data.index[-1], color='black', linestyle='--', alpha=0.8, label='Split Point')
plt.title('Train/Test Split for NYC Taxi Data')
plt.xlabel('Date')
plt.ylabel('Number of Taxi Trips')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Baseline Models

In [ ]:
# 3.1 Naive Forecaster (last value)
naive_model = NaiveForecaster()
naive_model.fit(train_data)
naive_forecast = naive_model.predict(len(test_data))

naive_mae = mean_absolute_error(test_data, naive_forecast)
naive_rmse = np.sqrt(mean_squared_error(test_data, naive_forecast))

print(f"Naive Model - MAE: {naive_mae:.2f}, RMSE: {naive_rmse:.2f}")

In [ ]:
# 3.2 Seasonal Naive Forecaster (daily seasonality)
seasonal_naive_model = SeasonalNaiveForecaster(season_length=48)  # 48 half-hours = 1 day
seasonal_naive_model.fit(train_data)
seasonal_naive_forecast = seasonal_naive_model.predict(len(test_data))

snaive_mae = mean_absolute_error(test_data, seasonal_naive_forecast)
snaive_rmse = np.sqrt(mean_squared_error(test_data, seasonal_naive_forecast))

print(f"Seasonal Naive Model - MAE: {snaive_mae:.2f}, RMSE: {snaive_rmse:.2f}")

## 4. Statistical Models

In [ ]:
# 4.1 ARIMA Model
print("Training ARIMA model...")
arima_model = ARIMAForecaster(order=(2, 1, 2))
arima_model.fit(train_data)
arima_forecast = arima_model.predict(len(test_data))

arima_mae = mean_absolute_error(test_data, arima_forecast)
arima_rmse = np.sqrt(mean_squared_error(test_data, arima_forecast))

print(f"ARIMA Model - MAE: {arima_mae:.2f}, RMSE: {arima_rmse:.2f}")

In [ ]:
# 4.2 Exponential Smoothing
print("Training Exponential Smoothing model...")
exp_smooth_model = ExponentialSmoothingForecaster(
    trend='add', seasonal='add', seasonal_periods=48
)
exp_smooth_model.fit(train_data)
exp_smooth_forecast = exp_smooth_model.predict(len(test_data))

exp_mae = mean_absolute_error(test_data, exp_smooth_forecast)
exp_rmse = np.sqrt(mean_squared_error(test_data, exp_smooth_forecast))

print(f"Exponential Smoothing Model - MAE: {exp_mae:.2f}, RMSE: {exp_rmse:.2f}")

## 5. Machine Learning Models

In [ ]:
# 5.1 Random Forest
print("Training Random Forest model...")
rf_model = RandomForestForecaster(
    lags=[1, 2, 3, 24, 48, 168], n_estimators=100
)
rf_model.fit(train_data)
rf_forecast = rf_model.predict(len(test_data))

rf_mae = mean_absolute_error(test_data, rf_forecast)
rf_rmse = np.sqrt(mean_squared_error(test_data, rf_forecast))

print(f"Random Forest Model - MAE: {rf_mae:.2f}, RMSE: {rf_rmse:.2f}")

In [ ]:
# 5.2 Feature-Rich ML Model using XGBoost
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

print("Training XGBoost model with rich features...")

# Prepare features
feature_cols = [col for col in train_ml.columns if col != 'value']
X_train = train_ml[feature_cols]
y_train = train_ml['value']
X_test = test_ml[feature_cols]
y_test = test_ml['value']

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)
xgb_model.fit(X_train_scaled, y_train)
xgb_forecast = xgb_model.predict(X_test_scaled)

xgb_mae = mean_absolute_error(y_test, xgb_forecast)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_forecast))

print(f"XGBoost Model - MAE: {xgb_mae:.2f}, RMSE: {xgb_rmse:.2f}")

## 6. Ensemble Model

In [ ]:
# Create ensemble of best performing models
print("Training Ensemble model...")

# Re-train individual models for ensemble
ensemble_arima = ARIMAForecaster(order=(2, 1, 2))
ensemble_exp = ExponentialSmoothingForecaster(trend='add', seasonal='add', seasonal_periods=48)
ensemble_rf = RandomForestForecaster(lags=[1, 2, 3, 24, 48, 168])

# Create ensemble with equal weights
ensemble_model = EnsembleForecaster(
    models=[ensemble_arima, ensemble_exp, ensemble_rf],
    weights=[0.33, 0.33, 0.34]
)

ensemble_model.fit(train_data)
ensemble_forecast = ensemble_model.predict(len(test_data))

ensemble_mae = mean_absolute_error(test_data, ensemble_forecast)
ensemble_rmse = np.sqrt(mean_squared_error(test_data, ensemble_forecast))

print(f"Ensemble Model - MAE: {ensemble_mae:.2f}, RMSE: {ensemble_rmse:.2f}")

## 7. Model Comparison

In [ ]:
# Compile all results
results = {
    'Model': ['Naive', 'Seasonal Naive', 'ARIMA', 'Exp. Smoothing', 'Random Forest', 'XGBoost', 'Ensemble'],
    'MAE': [naive_mae, snaive_mae, arima_mae, exp_mae, rf_mae, xgb_mae, ensemble_mae],
    'RMSE': [naive_rmse, snaive_rmse, arima_rmse, exp_rmse, rf_rmse, xgb_rmse, ensemble_rmse]
}

results_df = pd.DataFrame(results)
results_df['MAPE'] = [
    np.mean(np.abs((test_data.values - forecast) / test_data.values)) * 100
    for forecast in [naive_forecast, seasonal_naive_forecast, arima_forecast, 
                    exp_smooth_forecast, rf_forecast, xgb_forecast, ensemble_forecast]
]

# Sort by MAE
results_df = results_df.sort_values('MAE')

print("MODEL PERFORMANCE COMPARISON")
print("=" * 50)
print(results_df.to_string(index=False, float_format='%.2f'))

In [ ]:
# Visualize model comparison
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# MAE comparison
bars1 = ax1.bar(results_df['Model'], results_df['MAE'], color='skyblue', edgecolor='navy')
ax1.set_title('Mean Absolute Error (MAE)')
ax1.set_ylabel('MAE')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, mae in zip(bars1, results_df['MAE']):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
             f'{mae:.0f}', ha='center', va='bottom')

# RMSE comparison
bars2 = ax2.bar(results_df['Model'], results_df['RMSE'], color='lightcoral', edgecolor='darkred')
ax2.set_title('Root Mean Square Error (RMSE)')
ax2.set_ylabel('RMSE')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3, axis='y')

for bar, rmse in zip(bars2, results_df['RMSE']):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 15,
             f'{rmse:.0f}', ha='center', va='bottom')

# MAPE comparison
bars3 = ax3.bar(results_df['Model'], results_df['MAPE'], color='lightgreen', edgecolor='darkgreen')
ax3.set_title('Mean Absolute Percentage Error (MAPE)')
ax3.set_ylabel('MAPE (%)')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3, axis='y')

for bar, mape in zip(bars3, results_df['MAPE']):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
             f'{mape:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 8. Forecast Visualization

In [ ]:
# Plot forecasts for top performing models
top_models = results_df.head(4)

forecasts_dict = {
    'ARIMA': arima_forecast,
    'Exp. Smoothing': exp_smooth_forecast,
    'Random Forest': rf_forecast,
    'Ensemble': ensemble_forecast
}

plot_forecast_comparison(
    train_data.iloc[-500:],  # Show last 500 points of training
    test_data,
    forecasts_dict,
    figsize=(18, 10)
)

In [ ]:
# Zoom into first week of test predictions
week_points = 48 * 7  # 7 days * 48 half-hours
test_week = test_data.iloc[:week_points]

forecasts_week = {
    model: forecast[:week_points] 
    for model, forecast in forecasts_dict.items()
}

plt.figure(figsize=(18, 8))
plt.plot(test_week.index, test_week.values, 'k-', linewidth=3, label='Actual', alpha=0.8)

colors = ['red', 'green', 'orange', 'purple']
for i, (model, forecast) in enumerate(forecasts_week.items()):
    plt.plot(test_week.index, forecast, '--', color=colors[i], linewidth=2, label=f'{model} Forecast')

plt.title('First Week Forecast Comparison - NYC Taxi Demand')
plt.xlabel('Date')
plt.ylabel('Number of Taxi Trips')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. Residual Analysis

In [ ]:
# Analyze residuals for best performing model
best_model_name = results_df.iloc[0]['Model']
print(f"Analyzing residuals for best model: {best_model_name}")

if best_model_name == 'ARIMA':
    best_forecast = arima_forecast
elif best_model_name == 'Exp. Smoothing':
    best_forecast = exp_smooth_forecast
elif best_model_name == 'Random Forest':
    best_forecast = rf_forecast
elif best_model_name == 'XGBoost':
    best_forecast = xgb_forecast
else:
    best_forecast = ensemble_forecast

plot_residuals(test_data.values, best_forecast, model_name=best_model_name)

## 10. Feature Importance (for XGBoost)

In [ ]:
# Feature importance for XGBoost model
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance')
plt.title('Top 15 Feature Importance - XGBoost Model')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("Top 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

## 11. Cross-Validation

In [ ]:
# Time series cross-validation using ModelSelector
print("Performing time series cross-validation...")

# Select a subset of models for CV (to save time)
cv_models = {
    'seasonal_naive': SeasonalNaiveForecaster(season_length=48),
    'arima': ARIMAForecaster(order=(2, 1, 2)),
    'exp_smoothing': ExponentialSmoothingForecaster(trend='add', seasonal='add', seasonal_periods=48),
    'random_forest': RandomForestForecaster(lags=[1, 2, 3, 24, 48])
}

selector = ModelSelector(models=cv_models)
cv_results = selector.cross_validate(
    df['value'], n_splits=5, test_size=48*7, metric='mae'  # 1 week test sets
)

# Summary of CV results
cv_summary = cv_results.groupby('model')['score'].agg(['mean', 'std']).round(2)
cv_summary = cv_summary.sort_values('mean')

print("\nCross-Validation Results (MAE):")
print("=" * 40)
print(cv_summary)

In [ ]:
# Visualize CV results
plt.figure(figsize=(12, 6))

models = cv_summary.index.tolist()
means = cv_summary['mean'].values
stds = cv_summary['std'].values

plt.bar(models, means, yerr=stds, capsize=5, alpha=0.7, color='lightblue', edgecolor='navy')
plt.xlabel('Model')
plt.ylabel('MAE')
plt.title('Cross-Validation Results - Model Performance')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (mean, std) in enumerate(zip(means, stds)):
    plt.text(i, mean + std + 20, f'{mean:.0f}±{std:.0f}', 
             ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 12. Summary and Recommendations

In [ ]:
print("NYC TAXI FORECASTING - FINAL RESULTS")
print("=" * 50)

print(f"🏆 Best Model: {results_df.iloc[0]['Model']}")
print(f"   • MAE: {results_df.iloc[0]['MAE']:.2f} trips")
print(f"   • RMSE: {results_df.iloc[0]['RMSE']:.2f} trips")
print(f"   • MAPE: {results_df.iloc[0]['MAPE']:.1f}%")

print(f"\n📊 Model Rankings by MAE:")
for i, row in results_df.iterrows():
    print(f"   {results_df.index.get_loc(i)+1}. {row['Model']}: {row['MAE']:.2f}")

print(f"\n🎯 Key Insights:")
baseline_mae = results_df[results_df['Model'] == 'Seasonal Naive']['MAE'].iloc[0]
best_mae = results_df.iloc[0]['MAE']
improvement = (baseline_mae - best_mae) / baseline_mae * 100

print(f"   • Best model improves {improvement:.1f}% over seasonal naive baseline")
print(f"   • Seasonal patterns are crucial (seasonal naive >> naive)")
print(f"   • ML models benefit from rich feature engineering")
print(f"   • Ensemble approaches can provide robust predictions")

print(f"\n🚀 Deployment Recommendations:")
print(f"   • Use {results_df.iloc[0]['Model'].lower()} for production forecasting")
print(f"   • Implement real-time model retraining (weekly/monthly)")
print(f"   • Monitor for concept drift and seasonal changes")
print(f"   • Consider external factors (weather, events, holidays)")
print(f"   • Implement prediction intervals for uncertainty quantification")

print(f"\n📈 Business Impact:")
avg_trips = df['value'].mean()
error_percentage = best_mae / avg_trips * 100
print(f"   • Average prediction error: ±{best_mae:.0f} trips ({error_percentage:.1f}%)")
print(f"   • Useful for capacity planning and resource allocation")
print(f"   • Can optimize driver deployment and reduce wait times")

## Next Steps

1. **Model Deployment**: Package the best model for production use
2. **Real-time Pipeline**: Set up automated data ingestion and prediction
3. **Monitoring**: Implement model performance tracking and drift detection
4. **Enhancement**: Add external features (weather, events, holidays)
5. **Scaling**: Extend to other taxi zones or cities
6. **Business Integration**: Connect predictions to operational systems